## Reproducibility when testing

In [89]:
# see https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(0)

# when using cuda
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

# Getting Data

In [9]:
import os
from pathlib import Path
import scipy.io  # for reading matlab matrix
import numpy as np

import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import openmesh as om

In [11]:
data_location = 'D:\Data\CLOTHING\Learning Shared Shape Space_shirt_dataset_rest'

In [56]:
# custom DataSet class
class ParametrizedShirtDataSet(Dataset):
    """
    For loading the data of "Learning Shared Shape Space.." paper
    """
    
    def __init__(self, root_dir, transform = None):
        """
        Args:
            root_dir (string): Directory with all the t-shirt examples as subfolders
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        self.root_path = Path(root_dir)
        
        # list of items = subfolders
        self.datapoints_names = next(os.walk(self.root_path))[1]
        
        # remove non-valid element
        self.datapoints_names.remove('P3ORMPBNJJAJ')
        
        
        # datapoint folder structure
        self.mesh_filename = 'shirt_mesh_r.obj'
        self.pattern_params_filename = 'shirt_info.txt'
        self.features_filename = 'visfea.mat'
        self.garment_3d_filename = 'shirt_mesh_r_tmp.obj'
        
    def update_transform(self, transform):
        """apply new transform when loading the data"""
        self.transform = transform
        
        
    def __len__(self):
        """Number of entries in the dataset"""
        return len(self.datapoints_names)
    
    
    def read_verts(self, datapoint_name):
        """features parameters from a given datapoint subfolder"""
        assert (self.root_path / datapoint_name / self.garment_3d_filename).exists(), datapoint_name
        
        mesh = om.read_trimesh(str(self.root_path / datapoint_name / self.garment_3d_filename))
        
        return mesh.points()
    
    
    def read_pattern_params(self, datapoint_name):
        """9 pattern size parameters from a given datapoint subfolder"""
        assert (self.root_path / datapoint_name / self.pattern_params_filename).exists(), datapoint_name
        
        # assuming that we need the numbers from the last line in file
        with open(self.root_path / datapoint_name / self.pattern_params_filename) as f:
            lines = f.readlines()
            params = np.fromstring(lines[-1],  sep = ' ')
        return params
    
    
    def __getitem__(self, idx):
        """Called when indexing: read the corresponding data. 
        Does not support list indexing"""
        
        if torch.is_tensor(idx):  # allow indexing by tensors
            idx = idx.tolist()
            
        datapoint_name = self.datapoints_names[idx]
        
        vert_list = self.read_verts(datapoint_name)
        
        # read the pattern parameters
        pattern_parameters = self.read_pattern_params(datapoint_name)
        
        sample = {'features': vert_list.ravel(), 'pattern_params': pattern_parameters, 'name': datapoint_name}
        
        if self.transform is not None:
            sample = self.transform(sample)
        
        return sample
        
        
    def save_prediction_batch(self, predicted_params, names):
        """Saves predicted params of the datapoint to the original data folder"""
        
        for prediction, name in zip(predicted_params, names):
            path_to_prediction = self.root_path / '..' / 'predictions' / name
            try:
                os.makedirs(path_to_prediction)
            except OSError:
                pass
            
            prediction = prediction.tolist()
            with open(path_to_prediction / self.pattern_params_filename, 'w+') as f:
                f.writelines(['0\n', '0\n', ' '.join(map(str, prediction))])
                print ('Saved ' + name)
    
    def feature_size(self):
        return 12252 * 3
        

In [30]:
# Custom transforms -- to tensor
class SampleToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    
    def __call__(self, sample):
        features, params = sample['features'], sample['pattern_params']
        
        return {
                'features': torch.from_numpy(features).float(), 
                'pattern_params': torch.from_numpy(params).float(), 
                'name': sample['name']
                }

In [77]:
# Custom transforms -- normalize
class NormalizeInputfeatures(object):
    """Convert ndarrays in sample to Tensors."""
    def __init__(self, mean_features, std_features):
        self.mean = mean_features
        self.std = std_features
    
    def __call__(self, sample):
        features = sample['features']
        
        return {
                    'features': torch.div((features - self.mean), self.std), 
                    'pattern_params': sample['pattern_params'], 
                    'name': sample['name']
                }

In [17]:
# Data Normalization?

def get_mean_std(dataloader):
    
    stats = { 'batch_sums' : [], 'batch_sq_sums' : []}
    
    for data in dataloader:
        batch_sum = data['features'].sum(0)
        stats['batch_sums'].append(batch_sum)

    mean_features = sum(stats['batch_sums']) / len(dataloader)
    
    for data in dataloader:
        batch_sum_sq = (data['features'] - mean_features.view(1, len(mean_features)))**2
        stats['batch_sq_sums'].append(batch_sum_sq.sum(0))
                        
    std_features = torch.sqrt(sum(stats['batch_sq_sums']) / len(dataloader))
    
    return mean_features, std_features
    

In [58]:
# test loading
dataset = ParametrizedShirtDataSet(Path(data_location), 
                                  SampleToTensor())

print (len(dataset))
print (dataset[100]['features'])
print (dataset[100]['features'].shape)
print (dataset[0]['pattern_params'].shape)
#print (dataset[1000])

#loader = DataLoader(dataset, 10, shuffle=True)

1049
tensor([-2.6803, 12.6441, -0.1044,  ...,  2.7306, 12.6700, -0.0931])
torch.Size([36756])
torch.Size([9])


In [59]:
# test if all elements are avaliable


for name in dataset.datapoints_names:
    if not (dataset.root_path / name / dataset.garment_3d_filename).exists():
        print (name)

In [64]:
# Normalization of features
mean, std = get_mean_std(loader)
print (mean, std)

dataset_normalized = ParametrizedShirtDataSet(Path(data_location), 
                                  transforms.Compose([
                                      SampleToTensor(), 
                                      NormalizeInputfeatures(mean, std)]))

print (dataset[1]['features'])
print (dataset_normalized[1]['features'])

tensor([0.4847, 1.7808, 2.5189, 2.8437, 4.7678, 6.1286, 6.0374, 6.0461, 6.0504,
        6.0500, 6.0500, 6.0500, 6.0501, 6.0495, 6.0338, 6.0290, 6.0191, 6.0103,
        6.0033, 5.9944, 5.9856, 5.9771, 5.9658, 5.9569, 5.9481, 5.9396, 5.9291,
        5.9197, 5.9068, 5.9005, 5.7688, 5.1725, 4.5954, 3.9547, 2.9973, 2.3022,
        2.2025, 2.1716, 2.1283, 2.0894, 2.0596, 2.0363, 2.0191, 2.0048, 1.9907,
        1.9804, 1.9743, 1.9572, 1.9458, 1.9341, 1.9225, 1.9199, 1.9182, 1.9112,
        1.9119, 1.9027, 1.9047, 1.8997, 1.9000, 1.9002, 1.8983, 1.8905, 1.8855,
        1.8806, 1.8821, 1.8798, 1.8794, 1.8862, 1.9059, 1.9212, 1.9365, 1.9526,
        1.9691, 1.9886, 2.0051, 2.0236, 2.0428, 2.0614, 2.0781, 2.0829, 2.1120,
        2.1466, 2.1775, 2.2130, 2.2419, 2.2715, 2.3000, 2.3252, 2.3484, 2.3717,
        2.3947, 2.4142, 2.4290, 2.4452, 2.4613, 2.4773, 2.5043, 2.3297, 1.7479,
        0.7207])
tensor([-0.3243, -0.3104, -0.3113, -0.3173, -0.3096, -0.3099, -0.3147, -0.3157,
        -0.3159, -0.316

# Defining a Net

In [34]:
import torch.nn as nn
import torch.functional as F

In [72]:
class ShirtfeaturesMLP(nn.Module):
    """MLP for training on shirts dataset. Assumes 100 features parameters used"""
    
    def __init__(self):
        super().__init__()
        
        # layers definitions
        self.sequence = nn.Sequential(
            nn.Linear(36756, 3000), 
            nn.ReLU(), 
            nn.Linear(3000, 300), 
            nn.ReLU(), 
            nn.Linear(300, 60),
            nn.ReLU(),
            nn.Linear(60, 9)
        )
    
    def forward(self, x_batch):
        #print (x_batch)
        
        return self.sequence(x_batch)

In [38]:
net = ShirtfeaturesMLP()

print (net)

ShirtfeaturesMLP(
  (sequence): Sequential(
    (0): Linear(in_features=36756, out_features=3000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3000, out_features=300, bias=True)
    (3): ReLU()
    (4): Linear(in_features=300, out_features=60, bias=True)
    (5): ReLU()
    (6): Linear(in_features=60, out_features=9, bias=True)
  )
)


# Training

## Setup

In [87]:
# Basic Parameters
batch_size = 64
epochs_num = 100
learning_rate = 0.00001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cuda:0


In [60]:
# Initial load
shirt_dataset = ParametrizedShirtDataSet(Path(data_location), 
                                  SampleToTensor())




In [41]:
# Data normalization
mean, std = get_mean_std(DataLoader(shirt_dataset, 100))
shirt_dataset = ParametrizedShirtDataSet(Path(data_location), 
                                  transforms.Compose([
                                      SampleToTensor(), 
                                      NormalizeInputfeatures(mean, std)]))

In [61]:
# Data load and split
valid_size = (int) (len(shirt_dataset) / 10)
# split is RANDOM. Might affect performance
training_set, validation_set = torch.utils.data.random_split(
    shirt_dataset, 
    (len(shirt_dataset) - valid_size, valid_size))

print ('Split: {} / {}'.format(len(training_set), len(validation_set)))

training_loader = DataLoader(training_set, batch_size, shuffle=True)
validation_loader = DataLoader(validation_set, batch_size)

Split: 945 / 104


## Training loop

In [91]:
# Training loop func

def fit(model, regression_loss, optimizer, train_loader):
    model.to(device)
    
    for epoch in range (epochs_num):
        model.train()
        running_loss = 0.0
        for i, batch in enumerate(training_loader):
            features, params = batch['features'].to(device), batch['pattern_params'].to(device)
            
            #with torch.autograd.detect_anomaly():
            preds = model(features)
            loss = regression_loss(preds, params)
            #print ('Epoch: {}, Batch: {}, Loss: {}'.format(epoch, i, loss))
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            
            # logging
            if i % 5 == 4:
                wb.log({'epoch': epoch, 'loss': loss})
        
        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[(regression_loss(model(features), params), len(batch)) for batch in validation_loader]
            )
            
        valid_loss = np.sum(losses) / np.sum(nums)
        print ('Epoch: {}, Validation Loss: {}'.format(epoch, valid_loss))
        wb.log({'epoch': epoch, 'valid_loss': valid_loss})
        

In [90]:
# Get all the definitions
# model
model = ShirtfeaturesMLP()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# loss function
regression_loss = nn.MSELoss()

In [74]:
## Loss Test
batch = next(iter(validation_loader))

with torch.no_grad():
    preds = model(batch['features'])

print (preds)
print(regression_loss(preds, batch['pattern_params']))

tensor([[ 0.2414, -0.0883,  0.0889,  0.6119,  0.0395,  0.7041,  0.4997,  0.1071,
          0.1445],
        [ 0.2458, -0.0847,  0.0844,  0.6072,  0.0382,  0.7028,  0.4876,  0.1052,
          0.1425],
        [ 0.2435, -0.0814,  0.0975,  0.6140,  0.0406,  0.6994,  0.4985,  0.1132,
          0.1411],
        [ 0.2485, -0.0854,  0.0849,  0.6087,  0.0388,  0.7054,  0.4902,  0.1085,
          0.1438],
        [ 0.2466, -0.0833,  0.0824,  0.6033,  0.0397,  0.7027,  0.4860,  0.1088,
          0.1436],
        [ 0.2420, -0.0882,  0.0969,  0.6158,  0.0396,  0.6982,  0.5048,  0.1130,
          0.1381],
        [ 0.2360, -0.0828,  0.0953,  0.6124,  0.0403,  0.6988,  0.4980,  0.1096,
          0.1417],
        [ 0.2473, -0.0974,  0.1052,  0.6177,  0.0400,  0.6986,  0.5152,  0.1132,
          0.1365],
        [ 0.2406, -0.0818,  0.0946,  0.6142,  0.0412,  0.7008,  0.4970,  0.1108,
          0.1440],
        [ 0.2397, -0.0852,  0.0829,  0.6091,  0.0420,  0.7060,  0.4942,  0.1057,
          0.1496],


In [77]:
# init Weights&biases run
#os.environ['WANDB_MODE'] = 'dryrun'

import wandb as wb
wb.init(name = "mesh input", project = 'Test-Garments-Reconstruction')

wb.watch(model, log='all')

In [ ]:
# train

fit(model, regression_loss, optimizer, training_loader)

print ("Finished training")

In [90]:
# loss on validation set
model.eval()
with torch.no_grad():
    valid_loss = sum([regression_loss(model(batch['features']), batch['pattern_params']) for batch in validation_loader]) 

print ('Validation loss: {}'.format(valid_loss))

Validation loss: 0.02655116282403469


# Save predictions for validation to file

In [93]:
model.eval()
with torch.no_grad():
    batch = next(iter(validation_loader))    # might have some issues, see https://github.com/pytorch/pytorch/issues/1917
    shirt_dataset_normalized.save_prediction_batch(model(batch['features']), batch['name'])

[1.2462974786758423, 0.7441388368606567, 0.9016931056976318, 0.6540197134017944, 1.1041333675384521, 1.2501575946807861, 1.2502353191375732, 0.8995957374572754, 0.9998902678489685]
Saved V9SUOXEHXVLI
[1.2463276386260986, 0.7441512942314148, 0.9017415046691895, 0.65404212474823, 1.1041481494903564, 1.2502005100250244, 1.2502684593200684, 0.8996169567108154, 0.9999186396598816]
Saved Y6TLQDYOKMKC
[1.2463206052780151, 0.7441489696502686, 0.901718258857727, 0.6540318727493286, 1.1041452884674072, 1.2501857280731201, 1.2502539157867432, 0.8996154069900513, 0.9999144673347473]
Saved M6PNUHQOZYEF
[1.246446132659912, 0.7442108392715454, 0.9018248319625854, 0.6540975570678711, 1.1042542457580566, 1.2503091096878052, 1.2503726482391357, 0.8996891975402832, 1.0000224113464355]
Saved X5WIOMUQOEJD
[1.2462490797042847, 0.744117021560669, 0.901656985282898, 0.6539962291717529, 1.1040825843811035, 1.250112533569336, 1.2501859664916992, 0.8995614051818848, 0.9998473525047302]
Saved E7YOJODRKYQY
[1.2464